In [1]:
from main import eliminate_junk_and_rename_cols, get_df_from_pdf_exam

In [2]:
with open("resultado1570777439.pdf", "rb") as file:
    content = file.read()

In [3]:
df = get_df_from_pdf_exam(content)

AttributeError: 'str' object has no attribute 'columns'

In [4]:
df

,ANALITOS,RES - 1570777439 - 24/03/2024,RES - 4430270983 - 08/01/2023,RES - 3500486158 - 11/02/2019,VALORES DE REFERÊNCIA
1,Eritrócitos,"4,64","4,28","4,21(*)","3,83 a 4,99 milhões/mm3"
2,Hemoglobina,"12,8","11,6(1)","11,6(*)(1)","11,7 a 14,9 g/dL"
3,Hematócrito,"39,8","35,8","35,2(*)","35,1 a 44,1 %"
4,Hemoglobina Corpuscular Média,"27,6(1)","27,1(1)","27,6(*)(1)","27,7 a 32,7 pg"
5,VCM,"85,8","83,6","83,6(*)","83,1 a 96,8 fL"
6,Concentração de Hemoglobina\nCorpuscular,"32,2","32,4","33,0(*)","32,0 a 35,2 g/dL"
7,RDW,"15,2(1)","15,6(1)","14,9(*)(1)","11,8 a 14,2 %"
8,Leucócitos,5.820,4.700,5.750(*),3.470 a 8.290/mm3
9,Neutrófilos,3.540,2.820,3.060(*),1.526 a 5.020/mm3
10,Eosinófilos,80,70,60(*),20 a 340/mm3
